In [8]:
import sys
import os
import boto3
import rasterio


from rasterio.merge import merge

# Import raster helpers
sys.path.insert(0, "../../../../GOSTrocks/src")

import GOSTrocks.rasterMisc as rMisc
from GOSTrocks.misc import tPrint

# Import GOST urban functions
sys.path.insert(1, "../../../src")

# Import local functions
from novelUrbanization import *

%load_ext autoreload
%autoreload 2

s3client = boto3.client("s3")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\wbg\Anaconda3\envs\urban_test\Lib\site-packages\scipy\__init__.py", line 86, in <module>
    from scipy._lib._ccallback import LowLevelCallable
  File "c:\wbg\Anaconda3\envs\urban_test\Lib\site-packages\scipy\_lib\_ccallback.py", line 1, in <module>
    from . import _ccallback_c
ImportError: cannot import name '_ccallback_c' from 'scipy._lib' (c:\wbg\Anaconda3\envs\urban_test\Lib\site-packages\scipy\_lib\__init__.py)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\wbg\Anaconda3\envs\urban_test\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
  File "C:\Users\WB411133\AppData\Local\Temp\ipykernel_19728\2942251588.py", line 19, in <module>
    from novelUrbanization import *
  File "c:\WBG\Work\Code\GOSTurban\notebooks\Implementations\URB_SEAU1_NovelUrbanization\novelUrbanization.py", line 19, in <module>
    import GOSTurban.urban_helper as helpe

# Mosaic the DoU layers

In [ ]:
bucket = "wbg-geography01"
prefix = "URBANIZATION/MR_Novel_Poverty/"

# Loop through the S3 bucket and get all the keys for files that are .tif
more_results = True
loops = 0
urban_tiff = {}

while more_results:
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket, Prefix=prefix, ContinuationToken=token
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        if ("FINAL_STANDARD_1KM" in res["Key"]) and (
            res["Key"].endswith("urban.tif") or res["Key"].endswith("urban_hd.tif")
        ):
            cur_pop = os.path.basename(res["Key"]).split("_")[1]
            cur_type = os.path.basename(res["Key"]).split("_")[-1].replace(".tif", "")
            cur_lyr = f"{cur_pop}_{cur_type}"
            cur_path = os.path.join("s3://", bucket, res["Key"])
            try:
                urban_tiff[cur_lyr].append(cur_path)
            except:
                urban_tiff[cur_lyr] = [cur_path]

In [ ]:
out_folder = "/home/wb411133/temp/urban_mosaics"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

for cur_lbl, rasters in urban_tiff.items():
    tPrint(cur_lbl)
    out_file = os.path.join(out_folder, f"{cur_lbl}_DoU_Africa_mosaic.tif")
    curD, profile = rMisc.merge_rasters(
        rasters, merge_method="max", boolean_gt_0=True, out_file=out_file
    )

In [ ]:
opened_tiffs = [rasterio.open(x) for x in hd_urban_tiff]
merged, out_transform = merge(opened_tiffs, method="max")
merged = ((merged > 0) * 1).astype("uint8")

# Create a new raster file with the merged data
metadata = opened_tiffs[0].meta.copy()
metadata.update(
    {
        "height": merged.shape[1],
        "width": merged.shape[2],
        "transform": out_transform,
        "dtype": "uint8",
    }
)
with rasterio.open(
    f"/home/wb411133/temp/{sel_pop}_combo_urban_hd.tif", "w", **metadata
) as dst:
    dst.write(merged)

# Mosaick data from PP

In [ ]:
bucket = "wbg-geography01"
prefix = "URBANIZATION/MR_Novel_Poverty/AAPPC/Delineations/"

# Loop through the S3 bucket and get all the keys for files that are .tif
more_results = True
loops = 0
db_tiffs = {}

while more_results:
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket, Prefix=prefix, ContinuationToken=token
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        if "1k" in res["Key"]:
            cur_path = os.path.join("s3://", bucket, res["Key"])
            cur_type = os.path.basename(cur_path).split("_")[2][:2]
            if cur_type in ["cc", "co", "ur"]:
                cur_pop = os.path.basename(cur_path).split("_")[1].split("d")[0]
                cur_label = f"{cur_pop}_{cur_type}"
                try:
                    db_tiffs[cur_label].append(cur_path)
                except:
                    db_tiffs[cur_label] = [cur_path]

In [ ]:
out_folder = "/home/wb411133/temp/urban_mosaics"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

for cur_lbl, rasters in urban_tiff.items():
    tPrint(cur_lbl)
    out_file = os.path.join(out_folder, f"{cur_lbl}_DB_Africa_mosaic.tif")
    curD, profile = rMisc.merge_rasters(
        rasters, merge_method="max", boolean_gt_0=True, out_file=out_file
    )